In [1]:
import pandas as pd

import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

import mlflow

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716230824406, experiment_id='1', last_update_time=1716230824406, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()

categorical = ['PULocationID', 'DOLocationID']

numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.473870771499952

In [11]:
# save the model creating the directory model
os.makedirs('models', exist_ok=True)

In [12]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
# traaing the model with mlflow

with mlflow.start_run(run_name='lasso-regression') as run:
    
    mlflow.set_tag('developer', 'wesley')
    
    mlflow.log_param('train-data', 'green_tripdata_2021-01.parquet')
    mlflow.log_param('val-data', 'green_tripdata_2021-02.parquet')
    
    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)


In [11]:
import xgboost as xgb

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll import scope

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
params = {
    'max_depth': 8,
    'learning_rate': 0.12764891891262573,
    'reg_alpha': 0.09393441578920651,
    'reg_lambda': 0.26860224283011674,
    'min_child_weight': 3.2827464743316277,
    'objective': 'reg:linear',
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
        mlflow.log_param("valid-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

In [14]:
with mlflow.start_run():

    best_params = {
        'max_depth': 8,
        'learning_rate': 0.12764891891262573,
        'reg_alpha': 0.09393441578920651,
        'reg_lambda': 0.26860224283011674,
        'min_child_weight': 3.2827464743316277,
        'objective': 'reg:linear',
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
        
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.21074
[1]	validation-rmse:10.37584
[2]	validation-rmse:9.68182
[3]	validation-rmse:9.11237
[4]	validation-rmse:8.64117
[5]	validation-rmse:8.25884
[6]	validation-rmse:7.94727
[7]	validation-rmse:7.69513
[8]	validation-rmse:7.48979


/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [01:32:26] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[9]	validation-rmse:7.32369
[10]	validation-rmse:7.18888
[11]	validation-rmse:7.07973
[12]	validation-rmse:6.99060
[13]	validation-rmse:6.91774
[14]	validation-rmse:6.85679
[15]	validation-rmse:6.80499
[16]	validation-rmse:6.76343
[17]	validation-rmse:6.72769
[18]	validation-rmse:6.69789
[19]	validation-rmse:6.67227
[20]	validation-rmse:6.65069
[21]	validation-rmse:6.63059
[22]	validation-rmse:6.61331
[23]	validation-rmse:6.59789
[24]	validation-rmse:6.58591
[25]	validation-rmse:6.57400
[26]	validation-rmse:6.56156
[27]	validation-rmse:6.55199
[28]	validation-rmse:6.54386
[29]	validation-rmse:6.53415
[30]	validation-rmse:6.52568
[31]	validation-rmse:6.52053
[32]	validation-rmse:6.51337
[33]	validation-rmse:6.50796
[34]	validation-rmse:6.50413
[35]	validation-rmse:6.49794
[36]	validation-rmse:6.49256
[37]	validation-rmse:6.48739
[38]	validation-rmse:6.48295
[39]	validation-rmse:6.47900
[40]	validation-rmse:6.47425
[41]	validation-rmse:6.46921
[42]	validation-rmse:6.46691
[43]	validation

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [01:32:36] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
